In [14]:
import dateparser
import pandas as pd
from os import walk
import os
from glob import glob

In [27]:
# dates_dirs = dict()
# dir_name = "frog_pics"
# for root, folders, files in walk(dir_name):
#     cond = len(root.split("/")) == 4 
#     cond &= "no frogs found" not in root 
#     cond &= "no frogs captured" not in root
#     cond &= "no files" not in root.lower()
#     cond &= "Individual Frogs" not in root
#     cond &= "id complete" not in root.lower()
#     if cond:
#         print(root)
#         dates_dirs[root] = folders, files

In [29]:
""" open whareorino excel and try to connect each row to one or more pictures """
frog_id_dir = "frog_id"
whareorino_excel_file = f"{frog_id_dir}/Whareorino frog monitoring data 2005 onwards CURRENT FILE - DOCDM-106978.xlsm"
grid_name = "Grid A"
excel_db = pd.read_excel(whareorino_excel_file, sheet_name=grid_name)

In [35]:
# remove invalid rows by date
valid_idx = excel_db["Date"].notna()
valid_idx &= excel_db["Date"] != "Date"

In [40]:
excel_db["Photo #"].unique()

array([1490.0, 1491.0, '1492-1493', '1494-1495', '1496-1497', 1498.0,
       1514.0, '1515-1516', 1517.0, 1518.0, '1519-1520', 1521.0,
       '1522-1523', 1524.0, 1525.0, '1526-1527', 1528.0, '1529-1530',
       1531.0, 1532.0, '1533-1534', 1535.0, 1536.0, 1537.0, '1538-1539',
       1541.0, '1542-1543', '1544-1545', 1546.0, '1547-1548', 1549.0,
       '1550-1551', 1552.0, 1553.0, 1554.0, 1555.0, 1556.0, 2467.0,
       2468.0, 2469.0, 2470.0, 2471.0, 2472.0, 2473.0, 2474.0, 2475.0,
       2476.0, 2477.0, 2478.0, 2479.0, 2480.0, 2481.0, 2482.0, 2483.0,
       2484.0, 2485.0, 2486.0, 2487.0, 2488.0, 2489.0, 2490.0, 2491.0,
       2492.0, 2493.0, 2494.0, '2495-2496', 2497.0, 2498.0, 2499.0,
       2500.0, 2501.0, 2502.0, 2503.0, '2504-2505', 2506.0, '2507-2508',
       2509.0, 2510.0, 2511.0, 2512.0, 2513.0, 2514.0, 2515.0, 2516.0,
       2517.0, 2518.0, 2519.0, 2520.0, 2521.0, 2522.0, 2523.0, 2524.0,
       2525.0, 2526.0, 2527.0, 2528.0, 2529.0, '2530-2531', 2532.0,
       2533.0, 2534.

In [17]:
path_list = list(glob(f"{dir_name}/**/*.[jJ][pP][gG]", recursive=True))

In [25]:
path_list = sorted(path_list, key=lambda x: os.path.basename(x))

In [26]:
for f in path_list:
    print(f"{os.path.basename(f)}\t{f}")

(0)000-6723.jpg	frog_pics/Grid D/Individual Frogs/353/(0)000-6723.jpg
(0)000-6723.jpg	frog_pics/Grid D/Session 8/10 March 2009/ID Complete/(0)000-6723.jpg
(0)000-8049.jpg	frog_pics/Pukeokahu Frog Monitoring/Session 5 - March 2009/17 March 2009/ID Complete/(0)000-8049.jpg
(0)000-8049.jpg	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/106/(0)000-8049.jpg
(00)00-8059.jpg	frog_pics/Pukeokahu Frog Monitoring/Session 5 - March 2009/18 March 2009/ID Complete/(00)00-8059.jpg
(00)00-8059.jpg	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/110/(00)00-8059.jpg
(00)_0-6773.jpg	frog_pics/Grid D/Individual Frogs/362/(00)_0-6773.jpg
(00)_0-6773.jpg	frog_pics/Grid D/Session 8/12 March 2009/ID Complete/(00)_0-6773.jpg
(1)000-8074.jpg	frog_pics/Pukeokahu Frog Monitoring/Session 5 - March 2009/19 March 2009/ID Complete/(1)000-8074.jpg
(1)011-P266-DSC_5723.JPG	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/266/(1)011-P266-DSC_5723.JPG
(1)10(1)-6592.jpg	frog_pics/Grid D/Individual Frogs/29

0000-8275.jpg	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/149/0000-8275.jpg
0000-8275.jpg	frog_pics/Pukeokahu Frog Monitoring/Session 13 - Nov 2013 - Mar 2014/26.01.2014/ID complete/0000-8275.jpg
0000-8283.jpg	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/170/0000-8283.jpg
0000-8283.jpg	frog_pics/Pukeokahu Frog Monitoring/Session 13 - Nov 2013 - Mar 2014/8.02.2014/ID complete/0000-8283.jpg
0000-8284.JPG	frog_pics/Grid C/Session 20/13.11.2017/ID complete/0000-8284.JPG
0000-8284.JPG	frog_pics/Grid C/Individual Frogs/1016/0000-8284.JPG
0000-8285.jpg	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/156/0000-8285.jpg
0000-8285.jpg	frog_pics/Pukeokahu Frog Monitoring/Session 13 - Nov 2013 - Mar 2014/8.02.2014/ID complete/0000-8285.jpg
0000-8289.JPG	frog_pics/Grid C/Session 20/13.11.2017/ID complete/0000-8289.JPG
0000-8289.JPG	frog_pics/Grid C/Individual Frogs/1021/0000-8289.JPG
0000-8289.jpg	frog_pics/Pukeokahu Frog Monitoring/Individual Frogs/42/0000-8289.jpg
0000-8289.j

0100-10591.JPG	frog_pics/Grid D/Session 21/28.11.2018/ID Complete/0100-10591.JPG
0100-10591.JPG	frog_pics/Grid D/Individual Frogs/1057/0100-10591.JPG
0100-10602.JPG	frog_pics/Grid D/Session 21/28.11.2018/ID Complete/0100-10602.JPG
0100-10602.JPG	frog_pics/Grid D/Individual Frogs/1061/0100-10602.JPG
0100-10617.JPG	frog_pics/Grid D/Session 21/28.11.2018/ID Complete/0100-10617.JPG
0100-10617.JPG	frog_pics/Grid D/Individual Frogs/1049/0100-10617.JPG
0100-10628.JPG	frog_pics/Grid D/Session 21/29.11.2018/ID Complete/0100-10628.JPG
0100-10628.JPG	frog_pics/Grid D/Individual Frogs/784/0100-10628.JPG
0100-10719.jpg	frog_pics/Grid D/Individual Frogs/1126/0100-10719.jpg
0100-10741.jpg	frog_pics/Grid D/Individual Frogs/1138/0100-10741.jpg
0100-10743.jpg	frog_pics/Grid D/Individual Frogs/1139/0100-10743.jpg
0100-10798.jpg	frog_pics/Grid D/Individual Frogs/1126/0100-10798.jpg
0100-110.JPG	frog_pics/Grid D/Session 2/15-2-06/0100-110.JPG
0100-111.JPG	frog_pics/Grid D/Session 2/15-2-06/0100-111.JPG
010

1000-2876.jpg	frog_pics/Grid B/Session 13/22 March 2012/ID Complete/1000-2876.jpg
1000-2880.jpg	frog_pics/Grid B/Individual Frogs/333/1000-2880.jpg
1000-2880.jpg	frog_pics/Grid B/Session 13/22 March 2012/ID Complete/1000-2880.jpg
1000-29.jpg	frog_pics/Grid A/Individual Frogs/27/1000-29.jpg
1000-29.jpg	frog_pics/Grid A/Session 1/Grid A Session 1/10 Nov 05/1000-29.jpg
1000-3012.jpg	frog_pics/Grid B/Session 16/19-11-2013/ID complete/1000-3012.jpg
1000-3012.jpg	frog_pics/Grid B/Individual Frogs/331/1000-3012.jpg
1000-3022.jpg	frog_pics/Grid B/Session 16/19-11-2013/ID complete/1000-3022.jpg
1000-3022.jpg	frog_pics/Grid B/Individual Frogs/58/1000-3022.jpg
1000-3023.jpg	frog_pics/Grid B/Session 16/20-11-2013/ID complete/1000-3023.jpg
1000-3023.jpg	frog_pics/Grid B/Individual Frogs/384/1000-3023.jpg
1000-3029.jpg	frog_pics/Grid B/Session 16/20-11-2013/ID complete/1000-3029.jpg
1000-3029.jpg	frog_pics/Grid B/Individual Frogs/362/1000-3029.jpg
1000-3032.jpg	frog_pics/Grid B/Session 16/20-11-2013

1100-10052.jpg	frog_pics/Grid D/Session 16/11.12.2013/ID complete/1100-10052.jpg
1100-10052.jpg	frog_pics/Grid D/Individual Frogs/761/1100-10052.jpg
1100-10061.jpg	frog_pics/Grid D/Session 16/12.12.2013/ID complete/1100-10061.jpg
1100-10061.jpg	frog_pics/Grid D/Individual Frogs/615/1100-10061.jpg
1100-10066.jpg	frog_pics/Grid D/Session 16/12.12.2013/ID complete/1100-10066.jpg
1100-10066.jpg	frog_pics/Grid D/Individual Frogs/802/1100-10066.jpg
1100-10069.jpg	frog_pics/Grid D/Session 16/12.12.2013/ID complete/1100-10069.jpg
1100-10069.jpg	frog_pics/Grid D/Individual Frogs/688/1100-10069.jpg
1100-10072.jpg	frog_pics/Grid D/Session 16/12.12.2013/ID complete/1100-10072.jpg
1100-10072.jpg	frog_pics/Grid D/Individual Frogs/778/1100-10072.jpg
1100-10074.jpg	frog_pics/Grid D/Session 16/12.12.2013/ID complete/1100-10074.jpg
1100-10074.jpg	frog_pics/Grid D/Individual Frogs/793/1100-10074.jpg
1100-10075.jpg	frog_pics/Grid D/Session 16/12.12.2013/ID complete/1100-10075.jpg
1100-10075.jpg	frog_pics/

...

IMG_3002.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3002.JPG
IMG_3002.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3002.JPG
IMG_3003.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3003.JPG
IMG_3003.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3003.JPG
IMG_3004.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3004.JPG
IMG_3004.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3004.JPG
IMG_3005.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3005.JPG
IMG_3005.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3005.JPG
IMG_3006.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3006.JPG
IMG_3006.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3006.JPG
IMG_3007.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3007.JPG
IMG_3007.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3007.JPG
IMG_3008.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3008.JPG
IMG_3008.JPG	frog_pics/Grid D/Session 3/6 Dec 06/IMG_3008.JPG
IMG_3009.JPG	frog_pics/Grid B/Session 21/19.12.2018/IMG_3009.JPG
IMG_3009.JPG	frog_pics/Grid D/Session 3/6 Dec 

IMG_4708.JPG	frog_pics/Grid A/Session 7/25 Nov 2008/IMG_4708.JPG
IMG_4709.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4709.JPG
IMG_4710.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4710.JPG
IMG_4711.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4711.JPG
IMG_4712.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4712.JPG
IMG_4713.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4713.JPG
IMG_4714.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4714.JPG
IMG_4715.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4715.JPG
IMG_4716.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4716.JPG
IMG_4717.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4717.JPG
IMG_4718.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4718.JPG
IMG_4719.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4719.JPG
IMG_4720.JPG	frog_pics/Grid A/Session 7/26 Nov 2008/IMG_4720.JPG
IMG_4725.JPG	frog_pics/Grid A/Session 7/27 Nov 2008/IMG_4725.JPG
IMG_4726.JPG	frog_pics/Grid A/Session 7/27 Nov 2008/IMG_4726.JPG
IMG_4727.JPG	frog_pics/Gr

IMG_6928.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6928.JPG
IMG_6929.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6929.JPG
IMG_6930.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6930.JPG
IMG_6931.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6931.JPG
IMG_6932.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6932.JPG
IMG_6933.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6933.JPG
IMG_6934.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6934.JPG
IMG_6935.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6935.JPG
IMG_6936.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6936.JPG
IMG_6937.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6937.JPG
IMG_6938.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6938.JPG
IMG_6939.JPG	frog_pics/Grid B/Session 12/17 Feb 2011/IMG_6939.JPG
IMG_6943.JPG	frog_pics/Grid D/Session 12/28 Feb 2011/IMG_6943.JPG
IMG_6944.JPG	frog_pics/Grid D/Session 12/28 Feb 2011/IMG_6944.JPG
IMG_6945.JPG	frog_pics/Grid D/Session 12/28 Feb 2011/IMG_6945.JPG
IMG_6946.J

IMG_9508.JPG	frog_pics/Grid A/Session 16/19-11-2013/IMG_9508.JPG
IMG_9509.JPG	frog_pics/Grid A/Session 16/19-11-2013/IMG_9509.JPG
IMG_9510.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9510.JPG
IMG_9511.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9511.JPG
IMG_9512.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9512.JPG
IMG_9513.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9513.JPG
IMG_9514.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9514.JPG
IMG_9515.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9515.JPG
IMG_9516.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9516.JPG
IMG_9517.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9517.JPG
IMG_9518.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9518.JPG
IMG_9519.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9519.JPG
IMG_9520.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9520.JPG
IMG_9521.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9521.JPG
IMG_9522.JPG	frog_pics/Grid B/Session 16/19-11-2013/IMG_9522.JPG
IMG_9523.JPG	frog_pics/Gr

__00-4489.jpg	frog_pics/Grid C/Session 5/5 Dec 07/ID Complete/__00-4489.jpg
__00-4516.jpg	frog_pics/Grid C/Individual Frogs/252/__00-4516.jpg
__00-4516.jpg	frog_pics/Grid C/Session 5/5 Dec 07/ID Complete/__00-4516.jpg
__00-4621.jpg	frog_pics/Grid C/Individual Frogs/297/__00-4621.jpg
__00-4621.jpg	frog_pics/Grid C/Session 6/4 March 2008/ID Completed/__00-4621.jpg
__00-4626.jpg	frog_pics/Grid C/Individual Frogs/234/__00-4626.jpg
__00-4626.jpg	frog_pics/Grid C/Session 6/4 March 2008/ID Completed/__00-4626.jpg
__00-4640.jpg	frog_pics/Grid C/Individual Frogs/166/__00-4640.jpg
__00-4640.jpg	frog_pics/Grid C/Session 6/4 March 2008/ID Completed/__00-4640.jpg
__00-467.jpg	frog_pics/Grid A/Session 10/15 Feb 2010/ID Complete/__00-467.jpg
__00-467.jpg	frog_pics/Grid A/Individual Frogs/136/__00-467.jpg
__00-4671.jpg	frog_pics/Grid C/Individual Frogs/234/__00-4671.jpg
__00-4671.jpg	frog_pics/Grid C/Session 6/5 March 2008/ID Completed/__00-4671.jpg
__00-4680.jpg	frog_pics/Grid C/Individual Frogs/166/

# ------

In [11]:
import io
import json
from multiprocessing import AuthenticationError

import pandas as pd
import numpy as np

from panoptes_client import Panoptes, Project
from pandas.api.types import is_object_dtype

In [1]:
# zoo_user = getpass.getpass("Enter your Zooniverse user")
zoo_user = "lioruzan"
# zoo_pass = getpass.getpass("Enter your Zooniverse password")
zoo_pass = "@vsYP1muGZ3Z1R&I"


# Connect to Zooniverse with your username and password
auth = Panoptes.connect(username=zoo_user, password=zoo_pass)

if not auth.logged_in:
    raise AuthenticationError("Your credentials are invalid. Please try again.")

# Connect to the Zooniverse project (our frog project # is 13355)
project = Project(13355)

# Get the export classifications
export = project.get_export("classifications")

# Save the response as pandas data frame
classifications = pd.read_csv(
    io.StringIO(export.content.decode("utf-8")),
)
classifications.to_excel("zooniverse_keypoint_annotations.xls")
classifications.to_csv("zooniverse_keypoint_annotations.csv")

# for operational sake keep just these columns
classifications = classifications[["subject_data", "annotations"]]

/var/folders/s9/12gqnpd134j4ft6c49pfgsgr0000gn/T/ipykernel_59450/3379051029.py:32: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  classifications.to_excel("zooniverse_keypoint_annotations.xls")


In [4]:
def get_subject_data(subject_data_string):
    return list(json.loads(subject_data_string).values())[0]

In [9]:
# filter rows with bad capture ids (2-3 tops)
classifications_df = classifications.join(
    pd.DataFrame(
        data=classifications.subject_data.map(get_subject_data).to_list())
).drop(columns=["subject_data", "retired", "frog_id"])
idx = classifications_df.capture.notna()
classifications_df = classifications_df[idx]
numeric_idx = classifications_df.capture.str.isnumeric()
classifications_df = classifications_df[numeric_idx].astype({"capture": np.int32})

In [10]:
classifications_df.columns

Index(['annotations', 'grid', 'capture', 'filename'], dtype='object')

In [26]:
""" open whareorino excel and try to connect each row to one or more pictures """
frog_id_dir = "/Users/lioruzan/Downloads/frog_id"
dbs = dict()
whareorino_excel_file = f"{frog_id_dir}/Whareorino frog monitoring data 2005 onwards CURRENT FILE - DOCDM-106978.xlsm"
grids = ["Grid A", "Grid B", "Grid C", "Grid D"]
for grid in grids:
    df = pd.read_excel(whareorino_excel_file, sheet_name=grid)
    if is_object_dtype(df["Capture #"]):
        int_idx = df["Capture #"].str.isnumeric().isna()
        df = df[int_idx]
        int_idx = df["Capture #"].notna()
        dbs[grid] =  df[int_idx].astype({"Capture #": np.int32})
    else:
        nan_idx = df["Capture #"].notna()
        dbs[grid] = df[nan_idx].astype({"Capture #": np.int32})

In [27]:
pukeokahu_excel_file = f"{frog_id_dir}/Pukeokahu Monitoring Data 2006 onwards - DOCDM-95563.xlsx.xls"
df = pd.read_excel(pukeokahu_excel_file, sheet_name="MR Data")
nan_idx = df["Capture #"].notna()
dbs['Pukeokahu Frog Monitoring'] = df[nan_idx].astype({"Capture #": np.int32})

In [28]:
# use "capture", "Capture #" columns to merge rows from Zooniverse data to excel db rows
dbs_items = list(dbs.items())
for grid, grid_df in dbs_items:
    cur_grid_idx = classifications_df.grid == grid
    joined = (classifications_df[cur_grid_idx]
              .merge(right=grid_df, right_on="Capture #", left_on="capture")
              .drop(columns=["capture", "capture #"])
             )
    dbs[grid] = joined

In [34]:
# make all column names lowercase
for _, grid_df in dbs.items():
    grid_df.rename(columns=str.lower, inplace=True)
dbs["Grid A"].columns

Index(['annotations', 'grid', 'filename', 'date', 'capture #', 'bag #',
       'time found', 'capture location (to nearest m2)', 'x axis', 'y axis',
       'habitat type', 'height above ground (cm)', 'searcher', 'age class',
       'sex', 'svl (mm)', 'weight (g)', 'notes', 'unnamed: 15', 'photo #',
       'photographer', 'photography notes', 'back left mark',
       'back right mark', 'face left mark', 'face right mark',
       'capture photo code', 'frog id #', 'recapture from previous session',
       'within session recapture', 'id notes', 'unnamed: 29'],
      dtype='object')

In [40]:
# common columns, used for interface
cols = set([x.lower() for x in list(dbs["Grid A"].columns)])
for k,v in dbs.items():
    cols &= set([x.lower() for x in list(v.columns)])

cols

{'annotations',
 'back left mark',
 'back right mark',
 'bag #',
 'capture #',
 'capture photo code',
 'date',
 'face left mark',
 'face right mark',
 'filename',
 'frog id #',
 'grid',
 'id notes',
 'notes',
 'photo #',
 'photography notes',
 'time found',
 'within session recapture'}